In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# 데이터 불러오기
train = pd.read_csv('./data/train.csv', index_col=0)
test = pd.read_csv('./data/test.csv', index_col=0)
sample_submission = pd.read_csv('./data/sample_submission.csv', index_col=0)

# Train 데이터의 타입을 Sample_submission에 대응하는 가변수 형태로 변환
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x : to_number(x, column_number))

# 모델에 적용할 데이터 셋 준비 
x = train.drop(columns=['type', 'type_num'], axis=1)
y = train['type_num']

x = x.drop(columns=['fiberID'], axis=1)
test_x = test.drop(columns=['fiberID'],axis=1)

x_name=x.columns
col_name=x_name

In [3]:
x=np.array(x)
y=np.array(y)
test_x=np.array(test_x)

In [4]:
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.9, shuffle=True ,random_state=0)

/home/lab21/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [5]:
# from sklearn.preprocessing import StandardScaler, RobustScaler

# scaler1=StandardScaler()
# #scaler1=RobustScaler()

# scaler1.fit(x_train)
# x_train=scaler1.transform(x_train)
# x_test=scaler1.transform(x_test)

# test_x =scaler1.transform(test_x)

In [6]:
from sklearn.preprocessing import StandardScaler, RobustScaler

# scaler1=RobustScaler()

# scaler1.fit(x_train)
# x_train=scaler1.transform(x_train)
# x_test=scaler1.transform(x_test)

#test_x =scaler1.transform(test_x)

scaler2=StandardScaler()

scaler2.fit(x_train)
x_train=scaler2.transform(x_train)
x_test=scaler2.transform(x_test)

test_x =scaler2.transform(test_x)

In [7]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Using TensorFlow backend.


In [8]:
x=x.astype('float32')
test_x=test_x.astype('float32')

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation

In [10]:
model=Sequential()

model.add(Dense(64,activation='elu',input_dim=20)) # input dimension
model.add(Dense(128,activation='elu'))
model.add(Dense(256,activation='elu'))
model.add(Dense(256,activation='elu'))
model.add(Dropout(0.5))
model.add(Dense(256,activation='elu'))
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(256,activation='elu'))
model.add(Dense(128,activation='elu'))
model.add(Dense(19,activation='softmax'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                1344      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)      

In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
histo = model.fit(x_train, y_train, batch_size= 256, epochs=90, verbose=2, validation_split=1./9.)
loss, acc = model.evaluate(x_test,y_test,batch_size= 256)
print('Loss:',loss,'Accuracy:',acc)

Instructions for updating:
Use tf.cast instead.
Train on 159992 samples, validate on 19999 samples
Epoch 1/90
 - 4s - loss: 0.8356 - acc: 0.7302 - val_loss: 0.6666 - val_acc: 0.7623
Epoch 2/90
 - 4s - loss: 0.6571 - acc: 0.7794 - val_loss: 0.5871 - val_acc: 0.7979
Epoch 3/90
 - 4s - loss: 0.6152 - acc: 0.7916 - val_loss: 0.5748 - val_acc: 0.7952
Epoch 4/90
 - 4s - loss: 0.5889 - acc: 0.7994 - val_loss: 0.6082 - val_acc: 0.8026
Epoch 5/90
 - 4s - loss: 0.5693 - acc: 0.8060 - val_loss: 0.7883 - val_acc: 0.7424
Epoch 6/90
 - 4s - loss: 0.5588 - acc: 0.8092 - val_loss: 0.7535 - val_acc: 0.7157
Epoch 7/90
 - 4s - loss: 0.5504 - acc: 0.8113 - val_loss: 0.6321 - val_acc: 0.7666
Epoch 8/90
 - 4s - loss: 0.5371 - acc: 0.8171 - val_loss: 0.6838 - val_acc: 0.7706
Epoch 9/90
 - 4s - loss: 0.5377 - acc: 0.8152 - val_loss: 0.5708 - val_acc: 0.8045
Epoch 10/90
 - 4s - loss: 0.5315 - acc: 0.8185 - val_loss: 0.4609 - val_acc: 0.8421
Epoch 11/90
 - 4s - loss: 0.5252 - acc: 0.8201 - val_loss: 0.6111 - va

In [13]:
y_pred = model.predict_proba(test_x)

# 제출 파일 생성
submission = pd.DataFrame(data=y_pred, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('./data/submission_data_10.csv', index=True)